In [1]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

/home/prawater/.local/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage

class DietarySummary(BaseModel):
    summary: str = Field(description="A high-level summary of the user's diet in the past 24 hours")
    deviations: str = Field(description="Any deviations from the user's stated goals")
    suggestions: str = Field(description="Suggestions for the user to meet their goals more effectively")

summary_parser = JsonOutputParser(pydantic_object=DietarySummary)

user_goals = "maintain a calorie intake of 1500 kcal per day while focusing on reducing sugar and fat consumption."

rows = [
    {
        "calories": "160 kcal",
        "carbohydrates": "6g",
        "cholesterol": "0mg",
        "dietary_fiber": "3.5g",
        "fats": "14g",
        "id": 1,
        "name": "Almonds",
        "proteins": "6g",
        "sodium": "0mg",
        "sugars": "1g",
        "timestamp": "2024-11-24 11:39:54",
    },
]

prompt_text = f"""
You are a professional dietitian assisting a user in analyzing their dietary intake over the past 24 hours. The user has the following fitness goals:
{user_goals}

Based on the data below, provide:
1. A high-level summary of the user's diet.
2. Any deviations from their stated goals.
3. Suggestions to help them better align with their goals.

Dietary intake data:
{rows}

Format your response in the following JSON structure:
{summary_parser.get_format_instructions()}
"""

combined_prompt = HumanMessage(content=[
    {
        "type": "text",
        "text": prompt_text
    }
])

response = model.invoke([combined_prompt])
parsed_response = summary_parser.parse(response.content)

print(parsed_response)

{'summary': 'Based on the provided data, your 24-hour dietary intake consists primarily of almonds and a small amount of lozenges.  The almond intake provides a moderate amount of calories, protein, and healthy fats, while the lozenges contribute minimal nutrients.', 'deviations': 'Your current calorie intake is significantly below your goal of 1500 kcal.  While the almond consumption provides healthy fats, the total fat intake needs further evaluation to determine if it aligns with your fat reduction goal.  The sugar intake appears low, aligning with your goal in that aspect.', 'suggestions': "To reach your 1500 kcal goal, you need to substantially increase your food intake throughout the day. Focus on nutrient-dense foods that are lower in fat and added sugar.  Consider incorporating more fruits, vegetables, lean proteins (chicken, fish, beans, lentils), and whole grains.  It's important to spread your calorie intake across multiple meals and snacks.  Keep a detailed food diary for a